In [5]:
# The function localize takes the following arguments:
#
# colors:
#        2D list, each entry either 'R' (for red cell) or 'G' (for green cell)
#
# measurements:
#        list of measurements taken by the robot, each entry either 'R' or 'G'
#
# motions:
#        list of actions taken by the robot, each entry of the form [dy,dx],
#        where dx refers to the change in the x-direction (positive meaning
#        movement to the right) and dy refers to the change in the y-direction
#        (positive meaning movement downward)
#        NOTE: the *first* coordinate is change in y; the *second* coordinate is
#              change in x
#
# sensor_right:
#        float between 0 and 1, giving the probability that any given
#        measurement is correct; the probability that the measurement is
#        incorrect is 1-sensor_right
#
# p_move:
#        float between 0 and 1, giving the probability that any given movement
#        command takes place; the probability that the movement command fails
#        (and the robot remains still) is 1-p_move; the robot will NOT overshoot
#        its destination in this exercise
#
# The function should RETURN (not just show or print) a 2D list (of the same
# dimensions as colors) that gives the probabilities that the robot occupies
# each cell in the world.
#
# Compute the probabilities by assuming the robot initially has a uniform
# probability of being in any cell.
#
# Also assume that at each step, the robot:
# 1) first makes a movement,
# 2) then takes a measurement.
#
# Motion:
#  [0,0] - stay
#  [0,1] - right
#  [0,-1] - left
#  [1,0] - down
#  [-1,0] - up

In [11]:
def move(p, motion):
    # initialized a new posterior distribution
    aux = [[ 0.0 for row in range(len(p[0]))] for col in range(len(p))]
    for i in range(len(p)):
        for j in range(len(p[i])): 
            # consider components of going backwords in time with cyclic array
            aux[i][j] = (p_move * p[(i - motion[0]) % len(p)][j - motion[1] % len(p[i])]) + (1.0-p_move)*p[i][j]
       
    return aux

In [12]:
def sense(p, colors, measurement):
    # initialized a new posterior distribution
    aux = [[ 0.0 for row in range(len(p[0]))] for col in range(len(p))]
    # iterate over all grid cells
    s = 0.
    for i in range(len(p)):
        for j in range(len(p[i])):       
            hit = (measurement == colors[i][j]) 
            # non-normalized posterior based on prior
            aux[i][j] = p[i][j] * (hit * sensor_right + (1.- hit) * (1.0-sensor_right))
            s+=aux[i][j]
            
    # normalization
    for i in range(len(aux)):
        for j in range(len(p[i])):       
            aux[i][j] /= s
            
    return aux

In [13]:
def localize(colors,measurements,motions,sensor_right,p_move):
    # initializes p to a uniform distribution over a grid of the same dimensions as colors
    pinit = 1.0 / float(len(colors)) / float(len(colors[0]))
    p = [[pinit for row in range(len(colors[0]))] for col in range(len(colors))]
    
    for k in range(len(measurements)):
        p = move(p, motions[k])    
        # obtain a new probility distribution
        p = sense(p, colors, measurements[k])
        
    return p

In [14]:
def show(p):
    rows = ['[' + ','.join(map(lambda x: '{0:.5f}'.format(x),r)) + ']' for r in p]
    print ('[' + ',\n '.join(rows) + ']')

In [16]:
# Test 1
colors = [['G', 'G', 'G'],
          ['G', 'R', 'G'],
          ['G', 'G', 'G']]
measurements = ['R']
motions = [[0,0]]
sensor_right = 1.0
p_move = 1.0
p = localize(colors, measurements, motions, sensor_right, p_move)
show(p) # output the final distribution

[[0.00000,0.00000,0.00000],
 [0.00000,1.00000,0.00000],
 [0.00000,0.00000,0.00000]]


In [18]:
# Test 2
colors = [['G', 'G', 'G'],
          ['G', 'R', 'R'],
          ['G', 'G', 'G']]
measurements = ['R']
motions = [[0,0]]
sensor_right = 1.0
p_move = 1.0
p = localize(colors, measurements, motions, sensor_right, p_move)
show(p) # output the final distribution

[[0.00000,0.00000,0.00000],
 [0.00000,0.50000,0.50000],
 [0.00000,0.00000,0.00000]]


In [19]:
# Test 3
colors = [['G', 'G', 'G'],
          ['G', 'R', 'R'],
          ['G', 'G', 'G']]
measurements = ['R']
motions = [[0,0]]
sensor_right = 0.8
p_move = 1.0
p = localize(colors, measurements, motions, sensor_right, p_move)
show(p) # output the final distribution

[[0.06667,0.06667,0.06667],
 [0.06667,0.26667,0.26667],
 [0.06667,0.06667,0.06667]]


In [20]:
# Test 4
colors = [['G', 'G', 'G'],
          ['G', 'R', 'R'],
          ['G', 'G', 'G']]
measurements = ['R', 'R']
motions = [[0,0], [0,1]]
sensor_right = 0.8
p_move = 1.0
p = localize(colors, measurements, motions, sensor_right, p_move)
show(p) # output the final distribution

[[0.03333,0.03333,0.03333],
 [0.13333,0.13333,0.53333],
 [0.03333,0.03333,0.03333]]


In [21]:
# Test 5
colors = [['G', 'G', 'G'],
          ['G', 'R', 'R'],
          ['G', 'G', 'G']]
measurements = ['R', 'R']
motions = [[0,0], [0,1]]
sensor_right = 1.0
p_move = 1.0
p = localize(colors, measurements, motions, sensor_right, p_move)
show(p) # output the final distribution

[[0.00000,0.00000,0.00000],
 [0.00000,0.00000,1.00000],
 [0.00000,0.00000,0.00000]]


In [22]:
# Test 6
colors = [['G', 'G', 'G'],
          ['G', 'R', 'R'],
          ['G', 'G', 'G']]
measurements = ['R', 'R']
motions = [[0,0], [0,1]]
sensor_right = 0.8
p_move = 0.5
p = localize(colors, measurements, motions, sensor_right, p_move)
show(p) # output the final distribution

[[0.02899,0.02899,0.02899],
 [0.07246,0.28986,0.46377],
 [0.02899,0.02899,0.02899]]


In [24]:
# test 7
colors = [['G', 'G', 'G'],
          ['G', 'R', 'R'],
          ['G', 'G', 'G']]
measurements = ['R', 'R']
motions = [[0,0], [0,1]]
sensor_right = 1.0
p_move = 0.5
p = localize(colors, measurements, motions, sensor_right, p_move)
show(p) # output the final distribution

[[0.00000,0.00000,0.00000],
 [0.00000,0.33333,0.66667],
 [0.00000,0.00000,0.00000]]


In [25]:
colors = [['R','G','G','R','R'],
          ['R','R','G','R','R'],
          ['R','R','G','G','R'],
          ['R','R','R','R','R']]
measurements = ['G','G','G','G','G']
motions = [[0,0],[0,1],[1,0],[1,0],[0,1]]
sensor_right = 0.7
p_move = 0.8
p = localize(colors,measurements,motions,sensor_right, p_move)
show(p) 

[[0.01106,0.02464,0.06800,0.04472,0.02465],
 [0.00715,0.01017,0.08697,0.07988,0.00935],
 [0.00740,0.00894,0.11273,0.35351,0.04066],
 [0.00911,0.00715,0.01435,0.04313,0.03643]]
